<img src="header.png" align="left"/>

# Anwendungsbeispiel Import of excel data and timeseries prediction

Das Ziel dieses Beispieles ist es die Arbeit mit Zeitreihen aus Excel Files zu zeigen und darauf ein Vorhersagemodell für Zeitreihen zu entwickeln. Der Datensatz beschreibt die Entwicklung von Flugpassagierzahlen einer Airline in der fernen Vergangenheit. Die Daten sind aber typisch für Daten wie sie KMUs (z.B. Verkaufszahlen) zu finden sind. 


Der Code für das Beispiel wurde aus [1] adaptiert. Weitergehende Informationen sind zum Beispiel in [2] zu finden. 

- [1] [https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/](https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/)
- [2] [https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa](https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa)


Zitierung Datensatz:
```
Box, G. E. P., Jenkins, G. M. and Reinsel, G. C. (1976) Time Series Analysis, Forecasting and Control. Third Edition. Holden-Day. Series G.
```


# Import der Module

In [ ]:
#
# Import der Module
#
import os
import logging
import openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


#
# Abdrehen von Fehlermeldungen
#
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=Warning)
simplefilter(action='ignore', category=RuntimeWarning)


#
# Tensorflow und Keras
#
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM


#
# Für GPU Support
#
tflogger = tf.get_logger()
tflogger.setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR )
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


#
# Einstellen der Grösse von Diagrammen
#
plt.rcParams['figure.figsize'] = [16, 9]


#
# Ausgabe der Versionen
#
print('working on keras version {} on tensorflow {} using sklearn {}'.format ( tf.keras.__version__, tf.version.VERSION, sklearn.__version__ ) )

# Konstanten

In [ ]:
#
# Konstanten für Dateien
#
excelData = 'data/airline_passengers.xlsx'

In [ ]:
data = pd.ExcelFile(excelData)
print(data.sheet_names)

In [ ]:
#
# Auslesen eines Tabellenblattes
#
df = data.parse('Tabellenblatt1')
df.info
df.head(10)

In [ ]:
df.tail()

# Ausschneiden von Daten in pandas dataframes

Ein sehr gutes Tutorial dazu kann hier gefunden werden: [https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/](https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/)

<img src="info.png" align="left"/> 

In [ ]:
#
# Ausschneiden der benötigten Daten
#
df_cut = df.iloc[3:146,1:3]

In [ ]:
df_cut.head()

In [ ]:
df_cut.columns = ['month','passengers']

In [ ]:
df_cut.head()

In [ ]:
df_cut['passengers'].plot()

# Konventionelle Arbeit mit solchen Zeitreihen

Es gibt eine Reihe von älteren Methoden um mit solchen Zeitreihen umzugehen und auch gute Ergebnisse bei der Vorhersage zu erreichen. Beispiele dazu sind hier zu finden:

- https://machinelearningmastery.com/time-series-trends-in-python/
- https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa


Die wesentliche Leistung dieser Verfahren ist die Berechnung und Verwendung von Parameters wie **Trend** und **Saisonalität**. Wir hoffen, dass unsere Modelle damit umgehen können, ohne dass wir uns explizit darum kümmern müssen.


<img src="info.png" align="left"/> 

In [ ]:
x_data = df_cut['passengers'].values
x_data = x_data.astype('float32')
x_data = np.reshape(x_data,(-1,1))
print(x_data)

In [ ]:
#
# Normalize the dataset
#
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
scaler.fit(x_data)

In [ ]:
print(scaler.data_max_)

In [ ]:
x_data = scaler.transform(x_data)
print(x_data)

In [ ]:
# split into train and test sets
train_size = int(len(x_data) * 0.80)
test_size = len(x_data) - train_size
train, test = x_data[0:train_size,:], x_data[train_size:len(x_data),:]
print(len(train), len(test))

# Erzeugen eines Trainingsdatensatzes mit Hilfe von sliding windows

Sliding windows werden aus einer Zeitreihe erzeugt, indem ein Fenster über die gesamte Zeitreihe gezogen wird und dabei jeweils die Daten im Fenster kopiert werden. Mehr Details dazu hier [https://towardsdatascience.com/ml-approaches-for-time-series-4d44722e48fe](https://towardsdatascience.com/ml-approaches-for-time-series-4d44722e48fe).

<img src="info.png" align="left"/> 

In [ ]:
#
# Transformation einer Zeitreihe in sliding windows mit einem label (y)
#
def createSlidingWindowsWithLabel(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[ i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
#
# Anwenden der sliding window Funktion
#
look_back = 1
trainX, trainY = createSlidingWindowsWithLabel(train, look_back)
testX, testY = createSlidingWindowsWithLabel(test, look_back)

In [ ]:
# 
# reshape input to be [samples, time steps, features]
#
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
#
# create and fit the LSTM network
#
def createLSTMModel():
    model = Sequential()
    model.add(LSTM(10, input_shape=(1, look_back)))
    model.add(Dense(1,activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
lstm_model = createLSTMModel()

In [ ]:
lstm_model.fit(trainX, trainY, epochs=40, batch_size=1, verbose=1)

In [ ]:
#
# Schätzung der Werte für train und test Daten
#
trainPredict = lstm_model.predict(trainX)
testPredict = lstm_model.predict(testX)

In [ ]:
#
# Rücktransformation der Schätzungen (scaler)
#
trainPredict = scaler.inverse_transform(trainPredict)
trainYi = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testYi = scaler.inverse_transform([testY])

In [ ]:
#
# Anzeige der geschätzten neuen Werte
#
plt.plot(testYi[0,0:])
plt.plot(testPredict[1:,0])
plt.show()

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainYi[0,0:-1], trainPredict[1:,0]))
print('train loss: %.3f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testYi[0,0:-1], testPredict[1:,0]))
print('test loss: %.3f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(x_data)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[:len(trainPredict)+0, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(x_data)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2):len(x_data)-2, :] = testPredict

In [ ]:
# plot baseline and predictions
plt.plot(scaler.inverse_transform(x_data), color='grey')
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

## Welche Verbesserungsschritte wären sonst noch möglich?

<img src="info.png" align="left"/> 